In [1]:
import pandas as pd
import altair as alt

/tmp/ipykernel_159100/2443617991.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Parse sequence names. This info is used for plotting
!grep '^>' apple.indels.fa | awk '{match($0, /id[0-9]+/); id=substr($0, RSTART+2, RLENGTH-2); match($0, /len[0-9]+/); len=substr($0, RSTART+3, RLENGTH-3); print substr($0,2) "\t" id "\t" len}' > apple.names.tsv
!grep '^>' orange.indels.fa | awk '{match($0, /id[0-9]+/); id=substr($0, RSTART+2, RLENGTH-2); match($0, /len[0-9]+/); len=substr($0, RSTART+3, RLENGTH-3); print substr($0,2) "\t" id "\t" len}' > orange.names.tsv

In [47]:
apple = pd.read_csv('apple.names.tsv',sep="\t",names=['seq','divergence','length'])

In [48]:
sorted_apple = list(apple.sort_values(by=['divergence','length'],ascending=True)['seq'])

## Prep lastz output

In [49]:
# Remove "%" from columns of lastz output
!awk 'BEGIN{FS=OFS="\t"} {gsub(/%/, "", $13); gsub(/%/, "", $15); print}' lastz.tsv > lz.tsv

In [58]:
lz = pd.read_csv('lz.tsv',sep='\t',names="score,name1,strand1,size1,zstart1,end1,name2,strand2,size2,zstart2,end2,identity_rat,identity,coverage_rat,cvrg".split(","))

In [59]:
lz.head()

,score,name1,strand1,size1,zstart1,end1,name2,strand2,size2,zstart2,end2,identity_rat,identity,coverage_rat,cvrg
0,93266,APPLE_id100_len1000,+,3000,1000,2003,ORANGE_id100_len1000,-,3011,1000,2014,1002/1003,99.9,1003/3000,33.4
1,184118,APPLE_id100_len2000,+,4000,1000,3000,ORANGE_id100_len2000,+,4029,1000,3029,1999/2000,100.0,2000/4000,50.0
2,459054,APPLE_id100_len5000,+,7000,1000,6029,ORANGE_id100_len5000,+,7080,1000,6110,5017/5029,99.8,5029/7000,71.8
3,918611,APPLE_id100_len10000,+,12000,1000,11001,ORANGE_id100_len10000,+,12176,1000,11177,9997/10001,100.0,10001/12000,83.3
4,89388,APPLE_id99_len1000,+,3000,1000,2007,ORANGE_id99_len1000,-,3019,1000,2027,995/1007,98.8,1007/3000,33.6


In [61]:
# Retain only target info
lz = apple.merge(lz, left_on='seq',right_on='name1',how='left')[['seq','divergence','length','identity','cvrg']]

In [62]:
lz

,seq,divergence,length,identity,cvrg
0,APPLE_id100_len1000,100,1000,99.9,33.4
1,APPLE_id100_len2000,100,2000,100.0,50.0
2,APPLE_id100_len5000,100,5000,99.8,71.8
3,APPLE_id100_len10000,100,10000,100.0,83.3
4,APPLE_id99_len1000,99,1000,98.8,33.6
...,...,...,...,...,...
119,APPLE_id71_len10000,71,10000,70.9,83.4
120,APPLE_id70_len1000,70,1000,68.8,32.2
121,APPLE_id70_len2000,70,2000,70.8,50.1
122,APPLE_id70_len5000,70,5000,71.0,71.9


In [63]:
lz['aligner']='lastz'

## Process minimap2 output

In [21]:
!cut -f 1,6,7,10,11 mmap.tsv > mm.tsv

In [75]:
mm = pd.read_csv('mm.tsv',sep='\t',names='q,t,t_len,matches,align_len'.split(','))

In [76]:
mm['identity']=mm['matches']/mm['align_len']
mm['cvrg']=mm['align_len']/mm['t_len']

In [77]:
mm = apple.merge(mm, left_on='seq',right_on='t',how='left')[['seq','divergence','length','identity','cvrg']]

In [78]:
mm['aligner']='minimap2'

In [79]:
mm['identity']=mm['identity']*100
mm['cvrg']=mm['cvrg']*100

In [80]:
mm.head()

,seq,divergence,length,identity,cvrg,aligner
0,APPLE_id100_len1000,100,1000,96.015936,33.466667,minimap2
1,APPLE_id100_len2000,100,2000,95.051954,50.525000,minimap2
2,APPLE_id100_len5000,100,5000,94.661151,72.514286,minimap2
3,APPLE_id100_len10000,100,10000,94.543309,84.758333,minimap2
4,APPLE_id99_len1000,99,1000,79.035433,33.866667,minimap2


## Process FastGa

In [115]:
!cut -f 1,6,7,10,11 fastga.tsv > fga.tsv

In [116]:
fga = pd.read_csv('fga.tsv',sep='\t',names='q,t,t_len,matches,align_len'.split(','))

In [117]:
fga.head()

,q,t,t_len,matches,align_len
0,ORANGE_id100_len1000,APPLE_id100_len1000,3000,1000,1005
1,ORANGE_id100_len2000,APPLE_id100_len2000,4000,2000,2014
2,ORANGE_id100_len5000,APPLE_id100_len5000,7000,5000,5040
3,ORANGE_id100_len10000,APPLE_id100_len10000,12000,10001,10089
4,ORANGE_id99_len1000,APPLE_id99_len1000,3000,995,1010


In [118]:
fga['identity']=fga['matches']/fga['align_len']
fga['cvrg']=fga['align_len']/fga['t_len']

In [119]:
fga.head()

,q,t,t_len,matches,align_len,identity,cvrg
0,ORANGE_id100_len1000,APPLE_id100_len1000,3000,1000,1005,0.995025,0.335000
1,ORANGE_id100_len2000,APPLE_id100_len2000,4000,2000,2014,0.993049,0.503500
2,ORANGE_id100_len5000,APPLE_id100_len5000,7000,5000,5040,0.992063,0.720000
3,ORANGE_id100_len10000,APPLE_id100_len10000,12000,10001,10089,0.991278,0.840750
4,ORANGE_id99_len1000,APPLE_id99_len1000,3000,995,1010,0.985149,0.336667


In [120]:
fga = apple.merge(fga, left_on='seq',right_on='t',how='left')[['seq','divergence','length','identity','cvrg']]

In [121]:
fga['aligner']='fastga'

In [122]:
fga['identity']=fga['identity']*100
fga['cvrg']=fga['cvrg']*100

In [123]:
fga.head()

,seq,divergence,length,identity,cvrg,aligner
0,APPLE_id100_len1000,100,1000,99.502488,33.500000,fastga
1,APPLE_id100_len2000,100,2000,99.304866,50.350000,fastga
2,APPLE_id100_len5000,100,5000,99.206349,72.000000,fastga
3,APPLE_id100_len10000,100,10000,99.127763,84.075000,fastga
4,APPLE_id99_len1000,99,1000,98.514851,33.666667,fastga


## Plotting


In [124]:
data = pd.concat([lz,mm,fga], ignore_index=True)

In [125]:
data[data['aligner']=='fastga']

,seq,divergence,length,identity,cvrg,aligner
248,APPLE_id100_len1000,100,1000,99.502488,33.500000,fastga
249,APPLE_id100_len2000,100,2000,99.304866,50.350000,fastga
250,APPLE_id100_len5000,100,5000,99.206349,72.000000,fastga
251,APPLE_id100_len10000,100,10000,99.127763,84.075000,fastga
252,APPLE_id99_len1000,99,1000,98.514851,33.666667,fastga
...,...,...,...,...,...,...
368,APPLE_id70_len1000,70,1000,NaN,NaN,fastga
369,APPLE_id70_len2000,70,2000,NaN,NaN,fastga
370,APPLE_id70_len5000,70,5000,85.221576,71.242857,fastga
371,APPLE_id70_len10000,70,10000,85.223678,18.441667,fastga


In [131]:
import altair as alt

alt.Chart(data).mark_bar().encode(
    x=alt.X('seq:O', sort = sorted_apple),
    y=alt.Y('identity:Q'),
    color='cvrg:Q',
    row='aligner:N'
).properties(
    width=1200,  # Set width of the chart
    height=200  # Set height of the chart
)

alt.Chart(...)

In [ ]:
source